# **Importing the Dataset from Kaggle**

In this section, we connect Google Drive to access the Kaggle API token stored within it.
Using the Kaggle API, we import the **NASA Website Data dataset**, which contains **HTTP web server logs** from the **NASA Kennedy Space Center (KSC)** for August 1995. Each record represents a single request made by an Internet host to the NASA website and includes details such as the **requesting host, timestamp, requested resource (URL), HTTP status code**, and **response size in bytes**.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d djaouadnm/nasa-website-data

Dataset URL: https://www.kaggle.com/datasets/djaouadnm/nasa-website-data
License(s): CC0-1.0
  0% 0.00/16.1M [00:00<?, ?B/s]
100% 16.1M/16.1M [00:00<00:00, 1.24GB/s]


In [5]:
!unzip -o nasa-website-data.zip

Archive:  nasa-website-data.zip
  inflating: nasa_aug95_c.csv        


# **Reading CSV File**

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LogsAnalysis").getOrCreate()

In [8]:
# Define the dataset path
path = "/content/nasa_aug95_c.csv"

# Read the CSV file into a Spark DataFrame
df = spark.read.csv(path, header=True, inferSchema=True)

# Display the first 5 rows of the DataFrame
df.show(5)

+-----------------+-------------------+--------------------+------+-------------+
|  requesting_host|           datetime|             request|status|response_size|
+-----------------+-------------------+--------------------+------+-------------+
|in24.inetnebr.com|1995-08-01 00:00:01|GET /shuttle/miss...|   200|       1839.0|
|  uplherc.upl.com|1995-08-01 00:00:07|      GET / HTTP/1.0|   304|          0.0|
|  uplherc.upl.com|1995-08-01 00:00:08|GET /images/ksclo...|   304|          0.0|
|  uplherc.upl.com|1995-08-01 00:00:08|GET /images/MOSAI...|   304|          0.0|
|  uplherc.upl.com|1995-08-01 00:00:08|GET /images/USA-l...|   304|          0.0|
+-----------------+-------------------+--------------------+------+-------------+
only showing top 5 rows



In [9]:
# Number of rows
df.count()

1569888

In [10]:
# To see basic statistics of numeric columns
df.describe().show()

+-------+--------------------+-----------------+-----------------+----------------+
|summary|     requesting_host|          request|           status|   response_size|
+-------+--------------------+-----------------+-----------------+----------------+
|  count|             1569888|          1569888|          1569888|         1555720|
|   mean|                NULL|             NULL|211.9429118510365|17244.9678759674|
| stddev|                NULL|             NULL|35.07107507102347|68244.0386594844|
|    min|         ***.novo.dk|                ?|              200|             0.0|
|    max|zzzzzzzz.mindspri...|huttle/countdown/|              501|       3421948.0|
+-------+--------------------+-----------------+-----------------+----------------+



In [11]:
# Print the schema (structure) of the DataFrame
df.printSchema()

root
 |-- requesting_host: string (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- request: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- response_size: double (nullable = true)



In [12]:
# Check column names with their data types
df.dtypes

[('requesting_host', 'string'),
 ('datetime', 'timestamp'),
 ('request', 'string'),
 ('status', 'int'),
 ('response_size', 'double')]

In [13]:
# Get unique values from the status column
df.select("status").distinct().show()

+------+
|status|
+------+
|   501|
|   500|
|   403|
|   404|
|   200|
|   304|
|   302|
|   400|
+------+



In short:
*   **200s** → Successful responses
*   **300s** → Redirection or cache-related
*   **400s** → Client-side errors
*   **500s** → Server-side errors

In [14]:
# To check for missing or null values in selected columns
from pyspark.sql.functions import col

def check_nulls(df):
    for c in df.columns:
        num_null = df.filter((col(c).isNull()) | (col(c) == "")).count()
        print(f"The column '{c}' has {num_null} null values.")

check_nulls(df)

The column 'requesting_host' has 0 null values.
The column 'datetime' has 0 null values.
The column 'request' has 0 null values.
The column 'status' has 0 null values.
The column 'response_size' has 14168 null values.


In [15]:
# Drop the rows where 'Sales' has null values
df = df.dropna(subset=["response_size"])

In [16]:
check_nulls(df)

The column 'requesting_host' has 0 null values.
The column 'datetime' has 0 null values.
The column 'request' has 0 null values.
The column 'status' has 0 null values.
The column 'response_size' has 0 null values.


# **Find the top 10 most requested pages (URLs) on the NASA website.**

In [17]:
from pyspark.sql import functions as F

df_page = df.withColumn('Page', F.regexp_extract(F.col('request'), r'GET (.*?) HTTP', 1))

df_page.groupBy("Page") \
    .agg(F.count("request").alias('Total_requests')) \
    .orderBy(F.desc('Total_requests')) \
    .show(10, truncate=False)

+---------------------------------------+--------------+
|Page                                   |Total_requests|
+---------------------------------------+--------------+
|/images/NASA-logosmall.gif             |96854         |
|/images/KSC-logosmall.gif              |75143         |
|/images/MOSAIC-logosmall.gif           |66977         |
|/images/USA-logosmall.gif              |66605         |
|/images/WORLD-logosmall.gif            |65986         |
|/images/ksclogo-medium.gif             |62304         |
|/ksc.html                              |43392         |
|/history/apollo/images/apollo-logo1.gif|37754         |
|/images/launch-logo.gif                |35066         |
|/                                      |29875         |
+---------------------------------------+--------------+
only showing top 10 rows



# **At what hour of the day does the NASA website receive the most traffic?**

In [18]:
df_hour = df.withColumn('Hour', F.hour(F.col('datetime')))

df_hour.groupBy('Hour') \
       .agg(F.count('request').alias('Total_requests')) \
       .orderBy(F.desc('Total_requests')) \
       .show()

+----+--------------+
|Hour|Total_requests|
+----+--------------+
|  15|        108645|
|  12|        104255|
|  13|        103666|
|  14|        100661|
|  16|         98680|
|  11|         94637|
|  10|         87558|
|  17|         80043|
|   9|         78117|
|  18|         66222|
|   8|         64939|
|  22|         60054|
|  20|         59318|
|  19|         58710|
|  21|         57425|
|  23|         53935|
|   0|         47392|
|   7|         47062|
|   1|         38109|
|   2|         31809|
+----+--------------+
only showing top 20 rows



# **Find the day of the week (Monday, Tuesday, etc.) with the highest total number of requests.**

In [19]:
df_day = df.withColumn('Day', F.date_format(F.col('datetime'), 'EEEE'))

df_day.groupBy('Day') \
       .agg(F.count('request').alias('Total_requests')) \
       .orderBy(F.desc('Total_requests')) \
       .show()

+---------+--------------+
|      Day|Total_requests|
+---------+--------------+
| Thursday|        301669|
|  Tuesday|        276317|
|Wednesday|        253680|
|   Friday|        232515|
|   Monday|        226156|
|   Sunday|        133032|
| Saturday|        132351|
+---------+--------------+



# **Find the total response size per day of the week to see which day consumes the most bandwidth.**

In [20]:
df_day.groupBy('Day') \
      .agg(F.sum('response_size').alias('Total_response_size')) \
      .orderBy(F.desc('Total_response_size')) \
      .show()

+---------+-------------------+
|      Day|Total_response_size|
+---------+-------------------+
| Thursday|       5.04513484E9|
|  Tuesday|      4.679266066E9|
|Wednesday|      4.217550565E9|
|   Friday|      4.087947003E9|
|   Monday|      3.803970385E9|
|   Sunday|      2.516940002E9|
| Saturday|      2.477532563E9|
+---------+-------------------+



# **Which page generated the highest total response size (in bytes) across all requests?**

In [21]:
df_page.groupBy('Page')\
      .agg(F.sum('response_size').alias('Total_response_size')) \
      .orderBy(F.desc('Total_response_size')) \
      .show()

+--------------------+-------------------+
|                Page|Total_response_size|
+--------------------+-------------------+
|/shuttle/missions...|      1.640444674E9|
|/shuttle/missions...|       1.00509884E9|
|/shuttle/missions...|       5.11296379E8|
|/shuttle/technolo...|       4.92453023E8|
|/shuttle/missions...|       4.63593461E8|
|/shuttle/missions...|       4.39216472E8|
|/shuttle/countdow...|        4.0740895E8|
|     /images/rss.gif|       3.24018977E8|
|/images/ksclogo-m...|        3.1993164E8|
|/shuttle/missions...|        3.0956846E8|
|           /ksc.html|       2.91706417E8|
|  /images/launch.gif|       2.86305212E8|
|/shuttle/missions...|       2.78768874E8|
|/shuttle/technolo...|       2.53152638E8|
|/shuttle/missions...|       2.45903068E8|
|/shuttle/missions...|       2.38252603E8|
|/shuttle/missions...|       2.23765612E8|
|/images/launchmed...|       2.16696546E8|
|/shuttle/missions...|       2.08793835E8|
|/shuttle/technolo...|       2.00201448E8|
+----------

# **Identify potential bot/bulk-download activity and quantify its impact.**

In this step, we **analyze each requesting host** (IP or domain) to identify potential bot or bulk-download activity. By grouping requests by host, we calculate metrics such as the **total number of requests**, **total data downloaded, number of errors, and approximate number of distinct pages visited**. These metrics help detect abnormal patterns, for example, **hosts making a very large number of requests or downloading excessive data** are likely automated scripts or bots.

In [23]:
df_hosts = df_page.groupBy('requesting_host').agg(
    F.count('*').alias('total_requests'),
    F.sum('response_size').alias('total_bytes'),
    F.sum(F.when(F.col('status') >= 400, 1).otherwise(0)).alias('error_count'),
    F.approx_count_distinct('Page').alias('approx_distinct_pages')
)

In [24]:
df_hosts.orderBy(F.desc('total_requests')).show(20, truncate=False)

+--------------------+--------------+-----------+-----------+---------------------+
|requesting_host     |total_requests|total_bytes|error_count|approx_distinct_pages|
+--------------------+--------------+-----------+-----------+---------------------+
|edams.ksc.nasa.gov  |6515          |5.5832306E7|0          |331                  |
|piweba4y.prodigy.com|4797          |7.6491025E7|0          |649                  |
|163.206.89.4        |4769          |7.0187337E7|0          |730                  |
|piweba5y.prodigy.com|4564          |9.8635303E7|0          |726                  |
|piweba3y.prodigy.com|4354          |9.0309823E7|0          |657                  |
|www-d1.proxy.aol.com|3862          |6.0706648E7|0          |722                  |
|www-b2.proxy.aol.com|3516          |5.7577609E7|0          |649                  |
|www-b3.proxy.aol.com|3431          |5.9206086E7|0          |639                  |
|www-c5.proxy.aol.com|3403          |6.2706573E7|0          |602            